1.python读取nc数据时要注意单位。例如ISIMIP的数据单位为kg/m2/s，在时间尺度上不能直接求和
2.	将nc数据转为array处理时，如果array存在大量nan值时，分位数的计算会出现问题，应先使用as_array
3.	写入nc文件值注意nc文件无法储存nan值，需要将其设置为1e+20。此外，判断是否为nan不能用==，要用isnan


In [1]:
import sys
print(sys.executable)


D:\anaconda\python.exe


In [ ]:
!conda info --envs

这是一段用来读nc，并且查看它的基本信息的代码

In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
from osgeo import gdal, osr, ogr

os.environ['PROJ_LIB'] = r'C:\Users\asus\Anaconda3\envs\gdal\Library\share\proj'
os.environ['GDAL_DATA'] = r'C:\Users\asus\Anaconda3\envs\gdal\Library\share'

file_path = data_path
ds = nc.Dataset(file_path)
print("NetCDF file details:")
print(f"File format: {ds.file_format}")
print("Dimensions:")
for dim in ds.dimensions.values():
    print(dim)
print("Variables:")
for var in ds.variables.values():
    print(var)

ds.close()

这是一段用来写nc的代码，需要一个nc模板

In [ ]:
def write_nc(origin_path, input_path, output_path):
    nf_origin = nc.Dataset(origin_path) #这个用来后续保存新文件的时候提供数据模板
    nf = np.array(nc.Dataset(input_path).variables['这是你的变量，从前面的读nc基本信息中获取'][:])
    model_name = re.split('_|\.',os.path.basename(input_path))[0]

    output_path = output_path
    new_NC = nc.Dataset(output_path, 'w', format='NETCDF4')
    new_NC.createDimension('time', 12) #这个你得自己看
    new_NC.createDimension('lat', 360) #这个你得自己看
    new_NC.createDimension('lon', 720) #这个你得自己看
    new_NC.createDimension('qtot', 12 * 360 * 720) #这个你得自己看
    new_NC.createVariable('time', 'u4', ("time"))
    new_NC.createVariable('lat', 'f', ("lat"))
    new_NC.createVariable('lon', 'f', ("lon"))
    new_NC.createVariable('qtot', 'f', ("time", "lat", "lon")) #这个你得自己看
    # 向变量中填充数据
    new_NC.variables['lat'][:] = nf_origin.variables['lat'][:]
    new_NC.variables['lon'][:] = nf_origin.variables['lon'][:]
    new_NC.variables['qtot'][:] = nf

    # 最后记得关闭文件
    new_NC.close()
    
def main():
    write_nc(origin_path, input_path, output_path)

main()

这个是用来画一画nc的代码

In [ ]:

def plot_nc(data_path):
    file_path = data_path
    ds = nc.Dataset(file_path)
    data_show = ds.variables['qtot'][0]#这个你得自己看

    data_show = np.squeeze(data_show)
    data_show[data_show == 1e+20] = np.nan#这个你得自己看



    # Extract latitude and longitude data
    lat = ds.variables['lat'][:]
    lon = ds.variables['lon'][:]
    extent = [lon.min(), lon.max(), lat.min(), lat.max()]

    # Normalize the color scale using a suitable maximum value
    max_val = np.nanpercentile(data_show, 99)  # 99th percentile to exclude outliers
    norm = colors.Normalize(vmin=0, vmax=max_val)

    # Plotting the domestic water withdrawal
    plt.figure(figsize=(12, 6))
    img = plt.imshow(data_show, cmap='viridis', norm=norm, extent=extent, aspect='auto', origin='lower')
    cbar = plt.colorbar(img, label='Total monthly runoff (m/month)')
    plt.title('Total monthly runoff Distribution')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')

    plt.show()
    
def main():
    plot_nc(data_path)

main()

In [ ]:
这个是用来nc转tiff的代码

In [ ]:
def nc2tif(data, Output_folder):
    tmp_data = nc.Dataset(data) 
    Lat_data = tmp_data.variables['lat'][:]
    Lon_data = tmp_data.variables['lon'][:]
    
    pre_data = tmp_data.variables['ind'][:] #这个你得自己看
    tmp_arr = np.asarray(tmp_data.variables['ind']) #这个你得自己看

    # 影像的左上角&右下角坐标
    Lonmin, Latmax, Lonmax, Latmin = [Lon_data.min(), Lat_data.max(), Lon_data.max(), Lat_data.min()]
    print(Lonmin, Latmax, Lonmax, Latmin)
    # 分辨率计算
    Num_lat = len(Lat_data)
    Num_lon = len(Lon_data)
    Lat_res = (Latmax - Latmin) / (float(Num_lat) - 1)
    Lon_res = (Lonmax - Lonmin) / (float(Num_lon) - 1)

    driver = gdal.GetDriverByName('GTiff')
    out_tif_name = Output_folder + '\\' + data.split('\\')[-1].split('.')[0] + '.tif' #这个你得自己看
    out_tif = driver.Create(out_tif_name, Num_lon, Num_lat, 1, gdal.GDT_Float32)
    # 设置影像的显示范围
    # Lat_re前需要添加负号
    #这里的geotransform中第一个参数改成你的数据的左上角x坐标，第四个参数改成你的数据的左上角y坐标
    geotransform = (-179.75, Lon_res, 0.0, 89.75, 0.0, -Lat_res) #这个你得自己看
    out_tif.SetGeoTransform(geotransform)
    # 定义投影
    prj = osr.SpatialReference()
    prj.ImportFromEPSG(4326)  # 这里用的是WGS1984
    out_tif.SetProjection(prj.ExportToWkt())
    # 数据导出
    out_tif.GetRasterBand(1).WriteArray(np.squeeze(tmp_arr))  # 将数据写入内存，此时没有写入到硬盘
    out_tif.FlushCache()  # 将数据写入到硬盘
    out_tif = None  # 关闭tif文件



def main():
    
    nc2tif(input_path, output_path)


main()

这是一个读取一个文件夹下所有文件的代码，注意，这里是按照储存顺序（00，01，02），而不是数字顺序

In [ ]:
file = r'E:\PCR_download\ISIMIP\airrww\ISIMIP2b'
all_path = []
for root,dirs,files in os.walk(file):
    for file in files:
        path = os.path.join(root,file)
        all_path.append(path)

这是一个按照某个给定的栅格，重采样另一个栅格的代码

In [ ]:
def resample(reference_file, original_file, output_path)    
    reference_data = rioxarray.open_rasterio(reference_file)
    if reference_data.rio.crs is None:
        print("Reference data is missing CRS.")
        # 设置CRS，这里以WGS 84（EPSG:4326）为例
        reference_data.rio.write_crs("EPSG:4326", inplace=True)
        
        output_path = output_path
        original_data = rioxarray.open_rasterio(original_file)
        # 检查是否有CRS
        if original_data.rio.crs is None:
            print("Original data is missing CRS.")
            # 设置CRS，这里以WGS 84（EPSG:4326）为例
            original_data.rio.write_crs("EPSG:4326", inplace=True)
        resampled_data = original_data.rio.reproject_match(reference_data)
        resampled_data.rio.to_raster(output_path)

这是一个用来从shp文件中投影、提取面积和大于70%的地区的代码

In [ ]:
import rasterio.mask
import pandas as pd
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import Proj, Transformer
from shapely.geometry import Polygon, MultiPolygon
from rasterio.features import rasterize
from rasterio.transform import Affine

import rasterio
import numpy as np

from rasterio.mask import mask

#
# 加载Shapefile
gdf = gpd.read_file(r'E:\PCR_download\country_border\gadm_level1_data.shp')

# 定义转换器 - 从WGS 84到Mollweide投影
transformer = Transformer.from_proj(
    'epsg:4326',  # 源坐标系统 WGS 84
    '+proj=moll +lon_0=0 +datum=WGS84',  # 目标坐标系统 Mollweide
    always_xy=True)

# 计算球面面积
def calculate_spherical_area(geom):
    area = 0
    if geom.is_empty:
        return area
    if isinstance(geom, MultiPolygon):
        # 如果是MultiPolygon，处理每个单独的Polygon
        for poly in geom.geoms:
            x, y = poly.exterior.coords.xy
            x2, y2 = transformer.transform(x, y)
            new_geom = Polygon(zip(x2, y2))
            area += new_geom.area
    else:
        # 处理单个Polygon
        x, y = geom.exterior.coords.xy
        x2, y2 = transformer.transform(x, y)
        new_geom = Polygon(zip(x2, y2))
        area = new_geom.area
    return area

gdf['spherical_area'] = gdf['geometry'].apply(calculate_spherical_area)

# 计算总面积和80%的面积阈值
total_area = gdf['spherical_area'].sum()
area_threshold = total_area * 0.7

# 初始化一个空的GeoDataFrame和累积面积变量
selected_countries = gpd.GeoDataFrame(columns=gdf.columns, crs=gdf.crs)
accumulated_area = 0

# 按面积从大到小排序并选择国家
for index, country in gdf.sort_values(by='spherical_area', ascending=False).iterrows():
    if accumulated_area < area_threshold:
        selected_countries = pd.concat([selected_countries, gdf[gdf.index == index]], ignore_index=True)
        accumulated_area += country['spherical_area']
    else:
        break

# 将选定国家的数据转换回EPSG:4326
selected_countries = selected_countries.to_crs(epsg=4326)

# 保存新的Shapefile
selected_countries.to_file(r"E:\PCR_download\country_border\top_70_percent_countries.shp", driver='ESRI Shapefile')
print("已保存选定国家的Shapefile，包含面积之和超过全球总面积70%的国家。")

这是一个从shp文件中删除某个属性的对象，并重新存储的代码

In [ ]:
# 加载已经存在的Shapefile
gdf = gpd.read_file(r"E:\PCR_download\country_border\top_70_percent_countries.shp")

# 删除南极洲的记录，其GID_0为'ATA'
gdf_filtered = gdf[gdf['GID_0'] != 'ATA']

# 保存更新后的Shapefile
gdf_filtered.to_file(r"E:\PCR_download\country_border\filter_top_70_percent_countries.shp", driver='ESRI Shapefile')
print("已保存更新后的Shapefile，已从数据中删除南极洲。")

这是一个用来上下翻转栅格数据的代码

In [ ]:
from osgeo import gdal
import os
import numpy as np

os.environ['PROJ_LIB'] = r'C:\Users\asus\Anaconda3\envs\gdal\Library\share\proj'
os.environ['GDAL_DATA'] = r'C:\Users\asus\Anaconda3\envs\gdal\Library\share'



def flip_rasters(input_folder, output_folder):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历文件夹中的所有文件
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif"):  # 假设文件是TIFF格式
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 打开原始栅格文件
            ds = gdal.Open(input_path)
            band = ds.GetRasterBand(1)
            nodata = band.GetNoDataValue()

            # 读取数据
            data = band.ReadAsArray()

            # 翻转数据
            flipped_data = np.flipud(data)

            # 创建输出栅格文件
            driver = gdal.GetDriverByName('GTiff')
            out_ds = driver.Create(output_path, ds.RasterXSize, ds.RasterYSize, 1, band.DataType)
            out_band = out_ds.GetRasterBand(1)

            # 写入翻转后的数据
            out_band.WriteArray(flipped_data)

            # 设置无数据值
            if nodata is not None:
                out_band.SetNoDataValue(nodata)

            # 设置地理变换和投影信息
            out_ds.SetGeoTransform(ds.GetGeoTransform())
            out_ds.SetProjection(ds.GetProjection())

            # 清理
            out_band.FlushCache()
            out_ds = None
            ds = None

# 使用示例
input_folder = r'E:\PCR_download\Huang\ind_month'
output_folder = r'E:\PCR_download\Huang\ind_month_flipped'
flip_rasters(input_folder, output_folder)